# Transcripción y Traducción de Videos

Este notebook combina la funcionalidad de transcripción de videos y traducción de subtítulos. Está diseñado para ejecutarse en Google Colab y procesar videos ubicados en la carpeta 'Videos' de tu Google Drive, incluyendo subcarpetas.

In [ ]:
# Instalación de dependencias
!pip install git+https://github.com/openai/whisper.git
!pip install ffmpeg-python
!pip install anthropic

In [ ]:
# Verificar la versión de Whisper
import whisper
print(f"Versión de Whisper: {whisper.__version__}")

In [ ]:
# Función simplificada para cargar el modelo Whisper
def cargar_modelo_whisper():
    try:
        model = whisper.load_model("base")
        print("Modelo Whisper cargado exitosamente.")
        return model
    except Exception as e:
        print(f"Error al cargar el modelo Whisper: {e}")
        return None

modelo = cargar_modelo_whisper()
if modelo is not None:
    print("Tipo de modelo:", type(modelo))
else:
    print("No se pudo cargar el modelo Whisper.")

In [ ]:
# Si el problema persiste, reinstalar Whisper
# Descomenta las siguientes líneas si es necesario
# !pip uninstall -y openai-whisper
# !pip install --upgrade --no-deps --force-reinstall openai-whisper

In [ ]:
# Importación de bibliotecas
import os
import glob
import re
import time
import random
import logging
from google.colab import drive
from tqdm.notebook import tqdm
import anthropic
from IPython.display import display, Javascript

# Constantes
MAX_SUBTITLE_LENGTH = 48
MAX_SUBTITLES_PER_PART = 10
RETRY_DELAY_BASE = 2
MAX_RETRIES = 5
PAUSE_BETWEEN_TRANSLATIONS = 3
PAUSE_BETWEEN_VIDEOS = 15

In [ ]:
# Función para mantener la sesión activa
def keep_awake():
    display(Javascript('''
        function ClickConnect() {
            console.log("Manteniendo la conexión activa");
            document.querySelector("colab-toolbar-button#connect").click();
        }
        setInterval(ClickConnect, 60000)
    '''))

keep_awake()

In [ ]:
# Montar Google Drive y configurar Anthropic API
drive.mount('/content/drive')

api_key = input("Ingrese su clave API de Anthropic: ")
os.environ["ANTHROPIC_API_KEY"] = api_key
client = anthropic.Anthropic(api_key=api_key)

# Configurar logging
logging.basicConfig(filename='transcribe_translate.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
# Funciones de utilidad
def format_timestamp(seconds: float) -> str:
    milliseconds = int(seconds * 1000)
    hours, milliseconds = divmod(milliseconds, 3600000)
    minutes, milliseconds = divmod(milliseconds, 60000)
    seconds, milliseconds = divmod(milliseconds, 1000)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"

def mejorar_subtitulos(subtitulos: str, max_length: int = MAX_SUBTITLE_LENGTH) -> str:
    partes = subtitulos.split('\n\n')
    subtitulos_mejorados = []
    for parte in partes:
        lineas = parte.split('\n')
        if len(lineas) >= 3:
            numero, tiempo, *texto = lineas
            texto = ' '.join(texto)
            texto_dividido = []
            while texto:
                if len(texto) <= max_length:
                    texto_dividido.append(texto)
                    break
                corte = texto.rfind(' ', 0, max_length)
                if corte == -1:
                    corte = max_length
                texto_dividido.append(texto[:corte].strip())
                texto = texto[corte:].strip()
            texto = '\n'.join(texto_dividido)
            subtitulos_mejorados.append(f"{numero}\n{tiempo}\n{texto}")
    return '\n\n'.join(subtitulos_mejorados)

def dividir_srt(contenido: str, max_subtitulos: int = MAX_SUBTITLES_PER_PART) -> list:
    subtitulos = re.split(r'\n\n+', contenido.strip())
    return [('\n\n'.join(subtitulos[i:i+max_subtitulos])) for i in range(0, len(subtitulos), max_subtitulos)]

def corregir_estructura_srt(texto_original: str, texto_traducido: str) -> str:
    subtitulos_originales = re.split(r'\n\n+', texto_original.strip())
    subtitulos_traducidos = re.split(r'\n\n+', texto_traducido.strip())
    subtitulos_corregidos = []
    
    for i, subtitulo_original in enumerate(subtitulos_originales):
        lineas_original = subtitulo_original.split('\n')
        try:
            if i < len(subtitulos_traducidos):
                lineas_traducidas = subtitulos_traducidos[i].split('\n')
                subtitulo_corregido = '\n'.join(lineas_original[:2])  # Mantener el número y los tiempos
                subtitulo_corregido += '\n' + '\n'.join(lineas_traducidas[2:] if len(lineas_traducidas) > 2 else lineas_original[2:])
            else:
                subtitulo_corregido = subtitulo_original
            subtitulos_corregidos.append(subtitulo_corregido)
        except Exception as e:
            logging.error(f"Error procesando el subtítulo {i}: {e}")
            subtitulos_corregidos.append(subtitulo_original)  # Mantener el original si falla
    return '\n\n'.join(subtitulos_corregidos)

def traducir_parte(parte: str, reintentos: int = MAX_RETRIES) -> str:
    for intento in range(reintentos):
        try:
            respuesta = client.messages.create(
                model="claude-3-5-sonnet-20240620",
                max_tokens=4000,
                temperature=0.2,
                system="""Eres un traductor experto especializado en inteligencia artificial, Python y ChatBots. Tu tarea es traducir el siguiente bloque de subtítulos del inglés al español, enfocándote en el tema de IA y ChatBot. Sigue estas reglas estrictamente:

1. Mantén EXACTAMENTE la misma estructura del .srt, incluyendo números de subtítulo y marcas de tiempo.
2. Traduce SOLO el texto del subtítulo, dejando intactos los números y las marcas de tiempo.
3. NO traduzcas los términos técnicos específicos de IA y ChatBot.
4. REEMPLAZA todas las muletillas y sonidos de relleno como "um", "uh", "eh", "ah", "oh", "este", etc., por un espacio en blanco. No los traduzcas.
5. Mejora la redacción para que sea más clara, precisa y técnica, aplicando razonamiento y manteniendo el significado original.
6. Asegúrate de que cada subtítulo tenga su correspondiente traducción.
7. NO agregues ni quites líneas.
8. Cuando encuentres números de línea de tiempo repetidos, elimina uno de ellos: el que esté más cerca de una línea de texto.
9. Si hay alguna parte que no puedas traducir, déjala en su idioma original.
10. Es CRUCIAL que mantengas el mismo número de subtítulos y la misma estructura que el texto original.""",
                messages=[{"role": "user", "content": parte}]
            )
            return ''.join([bloque.text for bloque in respuesta.content])
        except anthropic.APIError as e:
            logging.warning(f"Error al traducir la parte: {e}. Reintentando {intento+1}/{reintentos}...")
            time.sleep((RETRY_DELAY_BASE ** intento) + (random.randint(0, 1000) / 1000))
        except Exception as e:
            logging.error(f"Error inesperado: {e}. Reintentando {intento+1}/{reintentos}...")
            time.sleep((RETRY_DELAY_BASE ** intento) + (random.randint(0, 1000) / 1000))
    raise Exception("Fallo al traducir la parte después de múltiples intentos.")

In [ ]:
# Función principal para transcribir y traducir
def transcribir_y_traducir(ruta_video: str) -> None:
    try:
        # Transcripción
        model = cargar_modelo_whisper()
        if model is None:
            raise Exception("No se pudo cargar el modelo Whisper")
        
        result = model.transcribe(ruta_video)

        subtitulos = ""
        for i, segmento in enumerate(result["segments"], start=1):
            subtitulos += f"{i}\n{format_timestamp(segmento['start'])} --> {format_timestamp(segmento['end'])}\n{segmento['text'].strip()}\n\n"

        subtitulos_mejorados = mejorar_subtitulos(subtitulos)

        # Guardar subtítulos originales
        ruta_base = os.path.splitext(ruta_video)[0]
        ruta_srt = f"{ruta_base}.srt"
        with open(ruta_srt, "w", encoding='utf-8') as archivo_subtitulos:
            archivo_subtitulos.write(subtitulos_mejorados)

        # Traducción
        partes = dividir_srt(subtitulos_mejorados)
        partes_traducidas = []

        for parte in tqdm(partes, desc="Traduciendo subtítulos"):
            parte_traducida = traducir_parte(parte)
            parte_corregida = corregir_estructura_srt(parte, parte_traducida)
            partes_traducidas.append(parte_corregida)
            time.sleep(PAUSE_BETWEEN_TRANSLATIONS)

        contenido_traducido = '\n\n'.join(partes_traducidas)

        # Guardar subtítulos traducidos
        ruta_srt_es = f"{ruta_base}.es.srt"
        with open(ruta_srt_es, "w", encoding='utf-8') as archivo_subtitulos:
            archivo_subtitulos.write(contenido_traducido)

        logging.info(f"Procesamiento completado para {ruta_video}")
    except Exception as e:
        logging.error(f"Error al procesar {ruta_video}: {e}")
        print(f"Error al procesar {ruta_video}: {e}")

# Procesamiento principal
def procesar_videos():
    ruta_base = '/content/drive/MyDrive/Videos'
    patron_videos = os.path.join(ruta_base, '**', '*.mp4')
    rutas_videos = glob.glob(patron_videos, recursive=True)

    for ruta_video in tqdm(rutas_videos, desc="Procesando videos"):
        transcribir_y_traducir(ruta_video)
        time.sleep(PAUSE_BETWEEN_VIDEOS)

# Ejecutar el procesamiento
procesar_videos()

print("Procesamiento completado. Revisa los archivos .srt y .es.srt en las carpetas correspondientes.")